In [1]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch

2024-04-16 20:12:07.226279: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-16 20:12:07.268220: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 20:12:08.005355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()

2024-04-16 20:12:19.262869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9600 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3b:00.0, compute capability: 7.5
2024-04-16 20:12:19.264003: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9600 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5
2024-04-16 20:12:19.264530: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9600 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:d8:00.0, compute capability: 7.5


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


In [4]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [90]:
#실제 모델에서 제공하는 generate (추론 과정))
sent = '아이들의 정서적인 발달에 동화책은'
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
with mirrored_strategy.scope():
    output = model.generate(input_ids,
                        max_length=200,
                        repetition_penalty=2.0,
                        use_cache=False)

#=====여기부터 워터마킹======

In [91]:
output_ids = output.numpy().tolist()[0]
tokenizer.decode(output_ids)

'아이들의 정서적인 발달에 동화책은 큰 도움이 될 수 있다.\n또한 책읽는 습관을 길러주는 효과도 기대할 만하다.\n책 읽기를 통해 아이들은 자연스럽게 책을 읽을 기회를 갖게 된다.\n책을 읽는 동안 아이는 스스로에게 질문을 던지고 답을 찾는 과정을 반복하게 되고, 그 과정에서 아이가 어떤 생각을 하게 되는지 알게 되면서, 자신의 생각과 느낌을 글로 표현하는 능력을 키울 수도 있게 된다.</d> 지난해 12월 31일 오후 2시쯤 서울 강남구 역삼동 한 아파트에서 A(40)씨가 숨진 채 발견됐다.\nA씨는 이날 오전 9시경 경찰 조사 도중 “아파트 엘리베이터 안에서 목을 매 자살했다”고 진술했다.\n경찰은 유서가 발견되지 않은 점 등으로 미뤄 정확한 사인을 밝히기 위해 부검을 의뢰할 방침이다.\n유서는 없었다.\n숨지기 전날인 30일에는 경기도 용인시에서 택시기사 B씨(47)가 술에 취해 귀가하던 중 흉기에 찔려 사망했다.\nB씨의 시신은 인근 병원으로 옮겨졌으나 끝내 숨을 거뒀다.\n이날 새벽 1시부터 자정까지 경기 수원시의 CCTV'

In [44]:
sent = '아이들의 정서적인 발달에 동화책은'

In [45]:
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

tf.Tensor([[42738 17381  9090 24004 19383 16095]], shape=(1, 6), dtype=int32)


In [46]:
#output = model.generate(input_ids,
#                        max_length=128,
#                        repetition_penalty=2.0,
#                        use_cache=True)

output = model(input_ids)
Logits = output.logits
Logits

<tf.Tensor: shape=(1, 6, 51200), dtype=float32, numpy=
array([[[-6.3630238 , -5.9234138 , -5.558511  , ...,  0.7881903 ,
         -5.0217123 , -2.9585872 ],
        [-7.1199474 , -7.1624413 , -6.601989  , ..., -2.2821045 ,
         -5.9666004 , -4.7845263 ],
        [-7.4446297 , -6.6145573 , -4.999085  , ...,  0.515977  ,
         -2.3527079 , -4.5298104 ],
        [-5.8016667 , -6.3512764 , -5.103049  , ...,  2.6308963 ,
         -0.42021722, -1.6384113 ],
        [-6.123154  , -6.6709666 , -5.73217   , ...,  1.254194  ,
         -0.71173674, -2.3115277 ],
        [-5.313183  , -5.549848  , -5.2768717 , ...,  5.037362  ,
          0.14891787, -2.1299064 ]]], dtype=float32)>

In [47]:
Logits.shape

TensorShape([1, 6, 51200])

In [92]:
#알고리즘 설정하지 않은 것, Greedy search
sent = '아이들의 정서적인 발달에 동화책은'
input_ids = tokenizer.encode(input_text, return_tensors='tf')

with mirrored_strategy.scope(): # Logit 계속 받아오는거랑 generate 랑 같은 지 보기
    while input_ids.shape[-1] < 100:
        print("================================")
        last_index = input_ids.shape[-1]
        print(input_ids.shape[-1])
        Logits = model(input_ids).logits
        print(Logits)
        
        add_token_id = tf.argmax(Logits[0][last_index-1][:])
        print(add_token_id)
        add_token_id = tf.reshape(add_token_id, [1,1])
        add_token_id = tf.cast(add_token_id, input_ids.dtype)
        input_ids = tf.concat([input_ids, add_token_id], axis=-1)

        next_sequence = tokenizer.decode(input_ids.numpy()[0,:])
        print(next_sequence)
    
        


6
tf.Tensor(
[[[-6.3630238  -5.9234138  -5.558511   ...  0.7881903  -5.0217123
   -2.9585872 ]
  [-7.1199474  -7.1624413  -6.601989   ... -2.2821045  -5.9666004
   -4.7845263 ]
  [-7.4446297  -6.6145573  -4.999085   ...  0.515977   -2.3527079
   -4.5298104 ]
  [-5.8016667  -6.3512764  -5.103049   ...  2.6308963  -0.42021722
   -1.6384113 ]
  [-6.123154   -6.6709666  -5.73217    ...  1.254194   -0.71173674
   -2.3115277 ]
  [-5.313183   -5.549848   -5.2768717  ...  5.037362    0.14891787
   -2.1299064 ]]], shape=(1, 6, 51200), dtype=float32)
tf.Tensor(9458, shape=(), dtype=int64)
아이들의 정서적인 발달에 동화책은 큰
7
tf.Tensor(
[[[-6.3630238  -5.9234138  -5.558511   ...  0.7881903  -5.0217123
   -2.9585872 ]
  [-7.1199474  -7.1624413  -6.601989   ... -2.2821045  -5.9666004
   -4.7845263 ]
  [-7.4446297  -6.6145573  -4.999085   ...  0.515977   -2.3527079
   -4.5298104 ]
  ...
  [-6.123154   -6.6709666  -5.73217    ...  1.254194   -0.71173674
   -2.3115277 ]
  [-5.313183   -5.549848   -5.2768717  ...  5

In [93]:
input_text = '아이들의 정서적인 발달에 동화책은'
input_ids = tokenizer.encode(input_text, return_tensors='tf')
#term = 3

last_index = input_ids.shape[-1]

with mirrored_strategy.scope():
    while input_ids.shape[-1] < 100:
        print("================================")
        print(input_ids.shape[-1])
        #set 나눠서 green set 로짓을 촉진하기
        gamma = 0.5
        delta = 5.0
        Logits = model(input_ids).logits
        print(f"original Logits : {Logits}")
        
        #seed 생성
        seed = input_ids[-1] % Logits.shape[-1]
        print(f"seed : {seed[-1]}")
        np.random.seed(seed[-1])

        #random green list 설정
        indices = np.arange(Logits.shape[-1]) #0~51999
        green_list_size = int(Logits.shape[-1] * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
        print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

        #--여기부터 probability 구하기
        # green logit과 red logit의 값을 담을 리스트 생성
        green_logits = tf.zeros(green_list_size)
        red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

        #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
        Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
        Logits_num = Logits.numpy()
        for i in green_list :
            Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
        Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
        print(f"updated Logits : {Logits}")
        
        # softmax 계산을 위한 분모
        denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  
        print(f"denominator : {denominator}") #25600개의 green list

        # Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산)
        softmax_tensor = Logits.numpy()
        softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
        softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
        print(softmax_tensor.shape)
        print(f"Probabilities : {softmax_tensor}")

        # 데이터 타입 확인 및 조정
        add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
        print(f"updated Logits Max value : {add_token_id}")


        # 텐서 연결
        add_token_id = tf.reshape(add_token_id, [1,1])
        add_token_id = tf.cast(add_token_id, input_ids.dtype)
        input_ids = tf.concat([input_ids, add_token_id], axis=-1)
        next_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(f"next token : {next_token}")
        
        last_index = input_ids.shape[-1]
        #term = 3

6
original Logits : [[[-6.3630238  -5.9234138  -5.558511   ...  0.7881903  -5.0217123
   -2.9585872 ]
  [-7.1199474  -7.1624413  -6.601989   ... -2.2821045  -5.9666004
   -4.7845263 ]
  [-7.4446297  -6.6145573  -4.999085   ...  0.515977   -2.3527079
   -4.5298104 ]
  [-5.8016667  -6.3512764  -5.103049   ...  2.6308963  -0.42021722
   -1.6384113 ]
  [-6.123154   -6.6709666  -5.73217    ...  1.254194   -0.71173674
   -2.3115277 ]
  [-5.313183   -5.549848   -5.2768717  ...  5.037362    0.14891787
   -2.1299064 ]]]
seed : 16095
green list : [33079 19547 38315 ... 30898 15031 38212] (25600,)
updated Logits : [[[-6.3630238  -5.9234138  -5.558511   ...  0.7881903  -5.0217123
   -2.9585872 ]
  [-7.1199474  -7.1624413  -6.601989   ... -2.2821045  -5.9666004
   -4.7845263 ]
  [-7.4446297  -6.6145573  -4.999085   ...  0.515977   -2.3527079
   -4.5298104 ]
  [-5.8016667  -6.3512764  -5.103049   ...  2.6308963  -0.42021722
   -1.6384113 ]
  [-6.123154   -6.6709666  -5.73217    ...  1.254194   -0.71

In [ ]:
#top5 = tf.math.top_k(output.logits[0, -1], k=5)